
# Latihan 01

Pada latihan ini, kita akan mencoba mencoba membedakan dua jenis gambar, gambar 5 dan s terbalik menggunakan Convolutional Neural Network (CNN) sederhana.

Latihan ini menggunakan teknik klasifikasi secara biner, alias hanya bisa membedakan 2 jenis kategori. Teknik untuk mengenali pattern gambar lebih dari 2 kategori akan dibahas di akhir kode.

## Install Library

Pertama, mari kita install library Augmentor terlebih dahulu

In [ ]:
!pip install Augmentor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Persiapan Dataset

Persiapankan dataset dengan memperbanyak masing-masing dataset train dan test dengan Augmentor.

In [ ]:
import Augmentor
import os

def perbanyak_(ini, sebanyak_ini):
  source_dir = ini
  output_dir = "."
  p = Augmentor.Pipeline(source_directory=source_dir, output_directory=output_dir)
  p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=1)
  p.rotate(probability=0.7, max_left_rotation=13, max_right_rotation=13)
  p.zoom_random(probability=0.5, percentage_area=0.9)
  p.crop_random(probability=0.6, percentage_area=0.9)
  p.resize(probability=1.0, width=64, height=64)

  p.sample(sebanyak_ini)

perbanyak_("/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Testing/Mobilbiasa", 300)
perbanyak_("/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Testing/Mobiltabrakan", 300)
perbanyak_("/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Testing/Mobilterbakar", 300)
perbanyak_("/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Training/Mobilbiasa", 700)
perbanyak_("/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Training/Mobiltabrakan", 700)
perbanyak_("/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Training/Mobilterbakar", 700)
perbanyak_("/content/drive/My Drive/Arista/MobilAmbulance/Testing", 300)
perbanyak_("/content/drive/My Drive/Arista/MobilAmbulance/Training", 700)


## Fase Training

Pada fase training ini, dipergunakan arsitektur CNN sederhana yang terdiri atas 4 layer konvolusi, 4 layer pooling, 1 layer flatten, yang terkoneksi dengan 2 fully connected layer lainnya.

Hasil akhir penentuan ada pada bagian node paling akhir di mana

`model.add(Dense(units = 1, activation = 'sigmoid'))`

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.metrics import categorical_accuracy

size_ = 64

model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape = (size_, size_, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 4, activation = 'softmax'))

model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

Berikut adalah arsitektur akhir CNN yang akan kita pergunakan.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)         7

Sekarang, mari kita set konfigurasi di mana lokasi data train dan data test kita berada.

In [ ]:
train = ImageDataGenerator(rescale = 1./255)
test = ImageDataGenerator(rescale = 1./255)
train_data = train.flow_from_directory('/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Training', target_size = (size_, size_), batch_size = 32, class_mode = 'categorical')
test_data = test.flow_from_directory('/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Testing', target_size = (size_, size_), batch_size = 32, class_mode = 'categorical')

Found 2891 images belonging to 4 classes.
Found 1254 images belonging to 4 classes.


Pastikan isi folder `dataset/train`nya hanya terdiri atas 2 folder saja, tanpa ada folder tambahan `.ipynb_checkpoints`.

Sekarang, mari kita train model kita menggunakan 10 kali epoch (perulangan).

In [ ]:
for i in range (0,240) :
  model.fit_generator(train_data, steps_per_epoch = 10, epochs = 1, validation_data = test_data, validation_steps = 10)
  model.save_weights('epoch_'+str(i)+'.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


10/10 [==============================] - 3s 299ms/step - loss: 0.0075 - categorical_accuracy: 1.0000 - val_loss: 3.4337 - val_categorical_accuracy: 0.6031


In [ ]:
model.save_weights("hasil.h5")

In [ ]:
model.load_weights("/content/drive/My Drive/Arista/Ibnu/H5/ambulance_148.h5")

Seperti yang bisa dilihat, jika model anda sudah benar, maka seharusnya nilai akurasi akan meningkat perlahan hingga mendekati 1.000 (alias 100%).

## Fase Testing

Pada fase ini, mari kita test keakuratan model yang sudah kita buat, apakah dia benar-benar bisa mengenali gambar `5` dan `s` dengan benar. Mari kita ambil 2 contoh file yang akan kita coba test keakuratan modelnya.



Karena hasil prediksi dalam bentuk angka, maka kita perlu membangun kamus, yang mana hasil prediksi tadi yang masih berupa angka, ditautkan dengan nama sebenarnya. Berikut adalah teknik membangun kamus tersebut.

In [ ]:
def switch_dict_key_values(this_dict):
    return dict((v,k) for k,v in this_dict.items())

nama_train_data = switch_dict_key_values(train_data.class_indices)
print(nama_train_data)

{0: 'Mobilambulance', 1: 'Mobilbiasa', 2: 'Mobiltabrakan', 3: 'Mobilterbakar'}


Sekarang terlihat bahwa `0` merepresentasikan `mobil ambulance`, `1` merepresentasikan `mobil biasa`, `2` merepresentasikan `mobil tabrakan`, dan `3` merepresentasikan `mobil terbakar`.

In [ ]:
from keras.preprocessing import image
import numpy as np
a = "/content/drive/MyDrive/Arista/Ibnu/Datasetmobil/Testing/Mobilterbakar/1599544787488.jpg"
img = image.load_img(a, target_size = (size_, size_))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

hasil = model.predict_classes(img)
print (hasil)
print("File gambar [", a, "] diprediksi sebagai", hasil, "alias terbaca sebagai :: ", nama_train_data[hasil[0]], ".")

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[3]
File gambar [ /content/drive/MyDrive/Arista/Ibnu/Datasetmobil/Testing/Mobilterbakar/1599544787488.jpg ] diprediksi sebagai [3] alias terbaca sebagai ::  Mobilterbakar .


Cek apakah ambulance terdeteksi ambulance

In [ ]:
import glob
from keras.preprocessing import image
import numpy as np

lol = glob.glob("/content/drive/My Drive/Arista/Ibnu/Datasetmobil/Testing/Mobilambulance/*.jpg")

total = 0
for haha in lol:
  img = image.load_img(haha, target_size = (size_, size_))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)

  hasil = model.predict_classes(img)
  #print("File gambar [s] diprediksi sebagai", hasil, "alias terbaca sebagai :: ", nama_train_data[hasil[0][0]], ".")
  total += (0 == hasil[0])*1
print(total/len(lol))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.7901234567901234
